In [213]:
import numpy as np
import warnings
import cv2

## Preparation


In [98]:
grainger_image = cv2.imread('grainger.jpg').astype(np.uint8)

## Naive Method: Increase blue channel

In [4]:
arr = grainger_image * np.array([0.5, 0.2, 0.1])
arr2 = (255 * arr/arr.max()).astype(np.uint8)

gamma = 2
gamma_img = np.array(255 * (arr2/255) ** gamma, dtype="uint8")
#window_name = 'image'
#cv2.imshow(window_name, gamma_img)
cv2.imwrite("night_final.png", gamma_img)

True

## Intermediate Method

In [210]:
def turn_day_to_night(image, alpha):
    image_width, image_height, color_channels = image.shape
    # Convert image to XYZ color space
    xyz_image = cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)

    # For the Scotopic Luminance of the image
    x = xyz_image[:,:,0]
    y = xyz_image[:,:,1]
    z = xyz_image[:,:,2]
    scotopic_luminance = y * (1.33 * (1 + (y + z)/x) - 1.68)
    scotopic_luminance = np.clip(np.nan_to_num(scotopic_luminance), 0, 255).astype(np.uint8)

    # Create a scotopic luminance image with 3 color channels
    scotopic_lum_img = np.zeros((image_width, image_height, color_channels)).astype(np.uint8)
    blue = np.array([.9, .5, .5]).astype(np.float64)
    k = 1
    for c in range(color_channels):
        scotopic_lum_img[:,:,c] = scotopic_luminance * blue[c] * k

    # Blend original and scotopic images pixel by pixel
    night_image = image * (1 - alpha) + scotopic_lum_img * alpha

    # Adjust contrast
    night_image = cv2.convertScaleAbs(night_image, alpha=0.8, beta=0)

    # Adding Gaussian blur to image
    kernel_size = 5
    night_image = cv2.GaussianBlur(night_image, (kernel_size, kernel_size), 2)

    #Adding noise to image
    noise_filter = np.zeros((image_width, image_height, color_channels), np.uint8)
    cv2.randu(noise_filter,(0),(10))
    
    result_image = night_image + noise_filter
    return result_image
    

In [214]:
warnings.filterwarnings('ignore')

alpha = 0.7
night_image = turn_day_to_night(grainger_image, alpha)
cv2.imwrite("night_final.png", night_image)

True

Sources used:
https://stackoverflow.com/questions/72888224/image-looks-overexposed-nearly-all-white-when-using-np-zeros-with-opencv-imsho

Papers referenced:
https://jamesferwerda.com/wp-content/uploads/2015/06/j09_thompson02_jgt.pdf